In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

from scipy.sparse import hstack, csr_matrix, vstack

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import f1_score

from keras.preprocessing import sequence 

C:\Users\duy.truong\AppData\Local\Continuum\anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
# Load data
train_df = pd.read_csv("./data/train.csv")
test_df = pd.read_csv("./data/test.csv")

In [3]:
train_df.head()

,id,comment,label
0,train_000000,Dung dc sp tot cam on \r\nshop Đóng gói sản ph...,0
1,train_000001,Chất lượng sản phẩm tuyệt vời . Son mịn nhưng...,0
2,train_000002,Chất lượng sản phẩm tuyệt vời nhưng k có hộp ...,0
3,train_000003,:(( Mình hơi thất vọng 1 chút vì mình đã kỳ vọ...,1
4,train_000004,Lần trước mình mua áo gió màu hồng rất ok mà đ...,1


In [4]:
train_df.shape

(16087, 3)

In [5]:
test_df.head()

,id,comment
0,test_000000,Chưa dùng thử nên chưa biết
1,test_000001,Không đáng tiềnVì ngay đợt sale nên mới mua n...
2,test_000002,Cám ơn shop. Đóng gói sản phẩm rất đẹp và chắc...
3,test_000003,Vải đẹp.phom oki luôn.quá ưng
4,test_000004,Chuẩn hàng đóng gói đẹp


In [6]:
test_df.shape

(10981, 2)

In [7]:
def count_nulls(df):
    null_counter = df.isnull().sum(axis=0)
    null_counter = null_counter[null_counter > 0]
    null_percent = df.isnull().sum(axis=0) / df.shape[0] * 100
    null_percent = null_percent[null_percent > 0]
    null_df = pd.concat([null_counter,null_percent],axis=1)
    null_df.columns = ['count','percent']
    display(null_df)

count_nulls(train_df)

,count,percent


In [39]:
vectorizer = TfidfVectorizer(max_features=100000,ngram_range=(1, 3))

In [40]:
x_train, x_val, y_train, y_val = train_test_split(train_df.comment, train_df.label, test_size=0.3,
    random_state=42)
x_train

2142      Chất lượng sản phẩm tuyệt vời \r\nRất đáng tiền.
3851                  Shop phục vụ rất kém giao thiếu hàng
5844                                          Thik lam\r\n
1082         Giao hàng kh đúng màu. Mặc dù đã chú thích.  
1671     Sản phẩm nhìn ko chất lượng giờ chạy ko đúng m...
8522                                  Giày như trong hình❤
10546                    Sản phẩm ok lắm. Nằm thấy dễ chịu
5145      Chất lượng sản phẩm tuyệt vời Đóng gói sản ph...
15380    Vỏ chặn bị dính bẩn như qua sd rôigiặt ko sạch...
1503          Võ ngoài bị lõm chắc do quá trình vận chuyển
1971                          Kim bị rớt... chất lượng kém
7372                                      "Thấy ok rồi đó"
3798     Shop giao hàng ko đúng 10 bộ size 6 mà có 4 bộ...
5508                         Loa dùng đc.giá phù hơp...
694       Chất lượng sản phẩm tuyệt vời không thích ngư...
12127           Hàng đẹp đóng cẩn thận nhưng giao hàng lâu
3855                      Trả tiền 2 mà mới nhận đc

In [41]:
vectorizer.fit(x_train)
x_tfidf_train = vectorizer.transform(x_train)
x_tfidf_val = vectorizer.transform(x_val)

In [42]:
x_tfidf_train.shape

(11260, 100000)

In [87]:
from sklearn.linear_model import LogisticRegression,RidgeClassifier
from sklearn.ensemble import GradientBoostingClassifier, GradientBoostingRegressor, RandomForestClassifier, ExtraTreesClassifier
from sklearn.metrics import accuracy_score, f1_score

model = GradientBoostingRegressor(
                                           n_estimators=500,
#                                            learning_rate= 0.002,
                                           max_depth = 7,
                                           min_samples_leaf=10,
                                           verbose =1)

# model = LogisticRegression()

print("Train model.......")
sentiment_fit = model.fit(X_train, y_train)
print("Predict ......")
y_pred = sentiment_fit.predict(X_test)

threshold = 0.5
y_pred = (y_pred > threshold).astype(np.uint8)

accuracy = accuracy_score(y_val, y_pred)
print("accuracy score: {0:.2f}%".format(accuracy*100))

f1_score = f1_score(y_val, y_pred)
print("F1 score: {0:.2f}".format(f1_score))

Train model.......
      Iter       Train Loss   Remaining Time 
         1           0.2323           13.18m
         2           0.2225           13.17m
         3           0.2142           13.10m
         4           0.2069           13.04m
         5           0.2010           12.75m
         6           0.1948           12.73m
         7           0.1896           12.56m
         8           0.1849           12.58m
         9           0.1808           12.46m
        10           0.1767           12.51m
        20           0.1481           13.10m
        30           0.1320           12.64m
        40           0.1199           12.52m
        50           0.1126           12.21m
        60           0.1063           11.94m
        70           0.0988           11.71m
        80           0.0946           11.45m
        90           0.0904           11.28m
       100           0.0860           11.25m
       200           0.0556            8.38m
       300           0.0398        

In [73]:
from keras.preprocessing import sequence 
from keras.preprocessing.text import Tokenizer

### Create sequence
vocabulary_size = 20000
tokenizer = Tokenizer(num_words= vocabulary_size)
tokenizer.fit_on_texts(x_train)

max_review_length = 5000
sequences = tokenizer.texts_to_sequences(x_train)
X_train = sequence.pad_sequences(sequences, max_review_length)

sequences = tokenizer.texts_to_sequences(x_val)
X_test = sequence.pad_sequences(sequences, max_review_length)

# X_train = x_tfidf_train
# X_test = x_tfidf_val

In [72]:
X_train.shape

(11260, 500)

In [76]:
# Build the model 
from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dense,Flatten, Dropout

top_words = 100000 
embedding_vector_length = 32

model = Sequential() 
model.add(Embedding(top_words, embedding_vector_length, input_length=max_review_length)) 
# model.add(LSTM(64))
# model.add(Dropout(0.2))
# model.add(LSTM(32))
model.add(Flatten()) 
model.add(Dense(1, activation='sigmoid')) 
model.compile(loss='binary_crossentropy',optimizer='adam', metrics=['accuracy']) 
print(model.summary()) 

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_9 (Embedding)      (None, 5000, 32)          3200000   
_________________________________________________________________
flatten_2 (Flatten)          (None, 160000)            0         
_________________________________________________________________
dense_9 (Dense)              (None, 1)                 160001    
Total params: 3,360,001
Trainable params: 3,360,001
Non-trainable params: 0
_________________________________________________________________
None


In [64]:
input_dim=(X_train.shape[0], X_train.shape[1])
           
model = Sequential()

model.add(LSTM(64, return_sequences=True, input_shape=input_dim))
model.add(Dropout(0.2))
model.add(LSTM(32))
model.add(Dense(1, activation="sigmoid"))
model.compile(loss='binary_crossentropy',optimizer='adam', metrics=['accuracy']) 
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_9 (LSTM)                (None, 11260, 64)         144640    
_________________________________________________________________
dropout_2 (Dropout)          (None, 11260, 64)         0         
_________________________________________________________________
lstm_10 (LSTM)               (None, 32)                12416     
_________________________________________________________________
dense_6 (Dense)              (None, 1)                 33        
Total params: 157,089
Trainable params: 157,089
Non-trainable params: 0
_________________________________________________________________
None


In [77]:
#Train the model
model.fit(X_train, y_train, validation_data=(X_test, y_val), nb_epoch=5, batch_size=64) 

C:\Users\duy.truong\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:2: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  


Train on 11260 samples, validate on 4827 samples
Epoch 1/5
11260/11260 [==============================] - 22s 2ms/step - loss: 0.5278 - acc: 0.7334 - val_loss: 0.3332 - val_acc: 0.8761
Epoch 2/5
11260/11260 [==============================] - 23s 2ms/step - loss: 0.2732 - acc: 0.8973 - val_loss: 0.2728 - val_acc: 0.8914
Epoch 3/5
11260/11260 [==============================] - 22s 2ms/step - loss: 0.2195 - acc: 0.9179 - val_loss: 0.2633 - val_acc: 0.8933
Epoch 4/5
11260/11260 [==============================] - 23s 2ms/step - loss: 0.1876 - acc: 0.9313 - val_loss: 0.2641 - val_acc: 0.8921
Epoch 5/5
11260/11260 [==============================] - 21s 2ms/step - loss: 0.1629 - acc: 0.9432 - val_loss: 0.2741 - val_acc: 0.8904


In [78]:
y_pred = model.predict(X_test)
y_pred

array([[4.0001981e-03],
       [9.6033317e-01],
       [6.9909960e-02],
       ...,
       [1.8958370e-01],
       [8.4179133e-01],
       [5.2799533e-07]], dtype=float32)

In [149]:
y_pred

array([[3.9474987e-03],
       [9.7973776e-01],
       [5.0319717e-03],
       ...,
       [1.3824444e-01],
       [8.8915288e-01],
       [4.1093484e-05]], dtype=float32)

In [79]:
from sklearn.metrics import accuracy_score, f1_score

threshold = 0.5
y_pred = (y_pred > threshold).astype(np.uint8)

accuracy = accuracy_score(y_val, y_pred)
print("accuracy score: {0:.2f}%".format(accuracy*100))

f1_score = f1_score(y_val, y_pred)
print("F1 score: {0:.2f}%".format(f1_score))

accuracy score: 89.04%
F1 score: 0.87%


In [146]:
from sklearn.metrics import accuracy_score, f1_score

y_pred = y_pred.astype(int)
f1 = f1_score(y_val, y_pred)
print("F1 score: {0:.2f}%".format(f1))

F1 score: 0.87%
